## Nous allons créer un Decision Tree permettant de classifier differents elements, en l'occurence des fruits en fonctions de trois attributs.

In [5]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [6]:
training_data = pd.read_csv("data.csv")

### Visualisez les donnees (valeurs reelles) ###

In [8]:
training_data.tail()

,Color,Size,Season,Label
95,Green,2,Winter,Kiwi
96,Yellow,3,Winter,Lemon
97,Red,3,Winter,Orange
98,Green,2,Winter,Kiwi
99,Green,2,Winter,Kiwi


### Definissez X de type np.array qui contiendra les valeurs des attributs ET des labels (pour faciliter le partinioning par la suite)

In [80]:
#X = np.matrix(training_data.label,training_data.season ).T
matriX = np.matrix([training_data.Color, training_data.Size, training_data.Season, training_data.Label]).T
X = np.array(training_data)

In [81]:
# Nous devons maintenant stocker les differents titres de categories pour pouvoir afficher notre arbre
header = list(training_data.columns)
header
X.shape

(100, 4)

In [82]:
X[2:4]

array([['Red', 3, 'Winter', 'Orange'],
       ['Green', 2, 'Winter', 'Kiwi']], dtype=object)

### Nous vous donnons la fonction permettant de trouver les differentes valeurs possibles pour chaque categorie, testez la pour mieux la comprendre 

In [75]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])
# La spécifité de l'objet set est qu'il ne peut pas contenir de doublons. Ses valeurs sont donc uniques.

### Testez votre fonction pour les differents attributs et labels ###

In [89]:
# Afficher les differentes saisons avec unique_vals
unique_vals(X[1:], header.index('Season'))

{'Spring', 'Summer', 'Winter'}

In [87]:
# Afficher les differentes couleurs avec unique_vals
unique_vals(X[1:], header.index('Color'))

{'Green', 'Red', 'Yellow'}

In [90]:
# Afficher les differents fruits avec unique_vals
unique_vals(X[1:], header.index('Label'))

{'Apple', 'Grape', 'Kiwi', 'Lemon', 'Orange'}

### Definir la fonction permettant de compter le nombre de fruits présents par categorie *(nombre de citrons, nombre de pommes, etc.)* 
#### La valeur de retour doit être un dictionnaire

In [130]:
#print(X[:,3])
def class_counts_head(rows, header_name): # pass unique_vals(X[1:], header.index('Label'))
    counts = {}
    col_index = header.index(header_name)
  
    counts = {value: list(rows[:,col_index]).count(value) \
              for value in set(unique_vals(X[1:], col_index))}
    return counts

 # list_to_count = unique_vals(rows, header.index(header_name))
   # #for fruit in header.index(header_name):
   # unique, counting = np.unique(rows, return_counts=True)
   # print (unique)
   # print (counting)
   # counts = (zip(unique, counting)) 
    # counts = {int(value): list(y).count(value) for value in set(y)}

def class_counts(rows): # pass unique_vals(X[1:], header.index('Label'))
    counts = {}
    for row in rows:
        counts.counter()[row] += 1
    # ...
    
    return counts

['Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Kiwi' 'Lemon' 'Grape' 'Apple'
 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape'
 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon'
 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi'
 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange'
 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple'
 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape'
 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon'
 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi'
 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange'
 'Kiwi' 'Lemon' 'Grape' 'Apple' 'Orange' 'Kiwi' 'Lemon' 'Orange' 'Kiwi'
 'Kiwi']


In [131]:
## Lancez votre fonction pour votre dataset :
count_label = class_counts_head(X[1:], 'Label')
## Elle devrait retourner : {'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20} 
print(count_label)
sum (count_label.values())

{'Grape': 18, 'Lemon': 20, 'Kiwi': 22, 'Orange': 20, 'Apple': 19}


99

### Definition de la fonction is_numeric permettant de savoir si une valeur est de type int/float ou non. ###

In [133]:
def is_numeric(value):
    if type(value) == int or type(value) == float:
        return True
    else:
        return False

In [138]:
## Testez votre fonction pour une valeur numerique (int ou float) ...
is_numeric(1)

True

In [139]:
#... et pour une valeur non-numérique 
is_numeric('Apple')

False

### Pour partitionner le dataset, nous allons utiliser une classe "Questions" ! Cette classe enregistre un numero de colonne (Ex : 0 pour la couleur) et une valeur de la colonne (ex : Vert).  
### La fonction match est utilisee pour comparer la valeur d'un exemple à celle de la question (voir commentaires fonction match).
#### Vous avez ici directement le constructeur et la fonction d'affichage. A vous de trouver la fonction match

In [184]:
class Question:
    
    def __init__(self, column, value):
        # Le Constructeur  permet d'attribuer les variables données à la classe
        self.column = column
        self.value = value

    def match(self, example):
        # Compare la valeur donné pour la question (value), a la valeur correspondante d'example 
        # (example est une ligne)
        # ATTENTION : La valeur de retour doit etre un boolean (True / False)
        
        return ( (self.value <= example[self.column]) \
                if (is_numeric(self.value) and is_numeric(example[self.column])) \
                else (self.value  == example[self.column]) )

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

#### Testez vos questions

In [207]:
# Ecrivez une question pour un attribut numerique
Y = X[1:2,3:4]
print(Y)
print (Y(2))
q1 = Question(1,3)
print(q1)
print(q1.value)
q1.match(["fff", 12, "ggg"])

[['Apple']]


TypeError: 'numpy.ndarray' object is not callable

In [182]:
# Et une deuxieme pour un attribut catégorique (couleurs / saisons) : 
q2 = Question(3, 'Apple')
print(q2)
q2.match(["fff", 5, "ggg", "Aple"])

Is Label == Apple?


False

In [ ]:
# Choisissez un exemple dans le dataset et vérifiez avec la fonction match s'il est de couleur rouge
example =
q3 = 

In [ ]:
# Pareil, mais tester pour size >= 2 (Attention à l'inegalité)
example =
q4 = 

### Vous devez definir la fonction partition qui prend en paramètres le dataset et une question, et les split en fonction de leur reponse (True / False) --> Ce seront nos 2 branches partant d'un noeud.

In [ ]:
def partition(rows, question):
    # Pour chaque ligne dans le dataset, partionnez le en 2 en fonction du resultat de la question.
    true_rows, false_rows = [], []
    
    # ...
    
    return true_rows, false_rows

In [ ]:
# Testons notre fonction partition pour isoler les lignes "rouges"
true_rows, false_rows = 

In [ ]:
# Et verifiez le contenue de vos 2 nodes


### Maintenant un peu de maths !
#### Nous allons calculer l'impurity d'une liste d'exemple grace a GINI 
#### ATTENTION : La fonction doit pouvoir prendre les listes qui suivent comme les lignes d'exemples contenues dans X (ex : gini(X[0]) ))

In [ ]:
def gini(rows):
    
    # ...
    
    return impurity

In [ ]:
# Pour verifier la fonction tester l'exemple suivant 
no_mixing = [['Apple'], ['Apple']]
# Que devrait-etre votre resultat ?
gini(no_mixing)

In [ ]:
 # Maintenant avec un autre
some_mixing = [['Apple'], ['Orange'], ['Orange'], ['Apple']]
#  Que devriez vous obtenir comme resultat ?
gini(some_mixing)

In [ ]:
# Et maintenant pour un dataset avec tous nos fruits
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Lemon'],
                  ['Kiwi']]
# Vous devriez obtenir environ 0.8
gini(lots_of_mixing)

### Calculons l'information gain 

In [ ]:
"""Information Gain :
The uncertainty of the starting node, minus the weighted impurity of
two child nodes.
"""

def info_gain(left, right, current_uncertainty):
    #left : true_rows
    #right : false_rows
    
    # ...
    
    return info_gain

In [ ]:
# Calculez l'uncertainy de notre dataset.
current_uncertainty = 

# Vous devriez obtenir environ 0.8

In [ ]:
# Quel est notre information gain du Dataset si nous partitionnons avec la question couleur == rouge ??
info_gain = 
# Resultat : environ 0.1835

In [ ]:
# De même pour la question "Size >= 2"
info_gain = 
# Resultat : environ 0.1925

### Pour ne pas tout tester manuellement, construisez la fonction vous permettant de trouver la question donnant le meilleur information gain en iterant sur chaque feature / value.

In [ ]:
def find_best_split(rows):
    best_gain = 0
    best_question = None  # 1 Objet Question

     # ...

    return best_gain, best_question

In [ ]:
### Testez votre fonction sur le dataset
best_gain, best_question = 
# Vous devriez avoir : is color == yellow ?

### Nous allons maintenant construire notre Decision Tree 
#### Les objets Leaf et Decision_Node sont donnés,  à vous de trouver la fonction qui permettra de construire l'arbre (build_tree)

In [ ]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Rappelez-vous de la condition d'arret et de la creation d'une leaf


### Fonction d'affichage de l'arbre (c'est cadeau ;) )

In [ ]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

### Maintenant, creez votre arbre et imprimez le ! ;)

In [ ]:
my_tree =

In [ ]:
print_tree(my_tree)

### Construisez la fonction classify qui permettra de parcourir l'abre pour un exemple donné. Elle nous retourne les predictions quand elle atteint une leaf (leaf.predictions)
#### Classify est une fonction recurssive allant de node en node :)
#### PS : man isinstance()

In [ ]:
def classify(row, node):


In [ ]:
# Test de la fonction avec le 16eme data sample. Vous devriez obtenir {'Kiwi': 22}
classify(X[15], my_tree)

In [ ]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [ ]:
# Maintenant avec une fonction d'affichage un peu plus plaisante. Resultat : {'Grape': '100%'}
print_leaf(classify(X[0], my_tree))

In [ ]:
# Tester avec ce dataset, puis essayer de changer les valeurs pour mieux comprendre le comportement de l'arbre
testing_data = [
    ['Green', 2, 'Winter','Kiwi'],
    ['Yellow', 3, 'Winter', 'Lemon'],
    ['Red', 1, 'Summer', 'Grape'],
    ['Red', 2, 'Summer', 'Grape'],
    ['Yellow', 3, 'Winter', 'Lemon'],
]

In [ ]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[3], print_leaf(classify(row, my_tree))))